### LOAD DATASET

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob 
import time
from datetime import datetime
from sklearn.model_selection import KFold, train_test_split
from sklearn.utils import shuffle

from keras.utils import np_utils
from keras.models import Sequential 
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

/home/ives/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### DATA PREPROCESSING 

In [2]:
img_h = 224
img_w = 224
def read_img(path,):
    'read, resize, and convert an image to grayscale'
    img = cv2.imread(path)
    resize = cv2.resize(img, (img_h, img_w), cv2.INTER_LINEAR)
    #gray = cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)
    return resize

In [3]:
type_paths = glob('/storage/Documents/springboard_capstone/capstone1/train/*')
test_paths = glob('/storage/Documents/springboard_capstone/capstone1/test/*')

In [4]:
train_paths = []
for path in type_paths:
    train_paths += glob(path+'/*')

In [5]:
def load_train(paths):
    train_data = []
    train_target = []
    train_id = []
    start_time = time.time()
    for path in paths:
        train_id.append(path)
        train_data.append(read_img(path))
        target = path.split('/')[-2]
        train_target.append(target)
    print ('Training data load time: {}'.format(time.time() - start_time))
    return train_data, train_target, train_id

def load_test(paths):
    test_data = []
    test_id = []
    start_time = time.time()
    for path in paths:
        test_id.append(path)
        test_data.append(read_img(path))
    print ('Testing data load time: {}'.format(time.time() - start_time))
    return test_data, test_id

In [6]:
train_data, train_target, train_id_ = load_train(train_paths)
test_data, test_id_ = load_test(test_paths)

Training data load time: 342.038950920105
Testing data load time: 123.54733157157898


In [7]:
def normalize_data(data):
    data = np.array(data, dtype=np.uint8)
    data = data.astype('float32')
    data = data/255
    return data

train_data = normalize_data(train_data)
print ('Shape of the training data is ', train_data.shape)

Shape of the training data is  (1481, 224, 224, 3)


In [8]:
test_data = normalize_data(test_data)
print ('Shape of the training data is ', test_data.shape)

Shape of the training data is  (503, 224, 224, 3)


In [9]:
# Convert train_target to one-hot-encoding before fitting into the model
train_label = [int(x.split('_')[1]) for x in train_target]
train_label = np_utils.to_categorical(train_label)
train_label = np.transpose(train_label)
train_label = np.transpose(train_label[~(train_label==0).all(axis=1)])

In [10]:
# Create validation split 
X_train, X_valid, Y_train, Y_valid = train_test_split(train_data, train_label, test_size=0.2, random_state=22)

### BUILDING MODEL

In [11]:
#Training hyperparameters
epochs = 50
batch_size = 16

In [12]:
#Early stopping callback
patience = 4
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0.05, 
                              patience = patience, verbose=0, mode='auto')

In [13]:
#Build the first model with a simple stack of 3 convolution layers with a ReLU activation and max_pooling

model1 = Sequential()
model1.add(Conv2D(32, (3, 3), input_shape=(img_h, img_w, 3), data_format='channels_last'))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(64, (3, 3), data_format='channels_last'))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(128, (3,3), data_format='channels_last'))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model1.add(Dropout(0.25))

#Add flatten
model1.add(Flatten())
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(3, activation='softmax'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model1.compile(loss='categorical_crossentropy', 
             optimizer=sgd,
             metrics = ['accuracy'])

In [14]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
activation_2 (Activation)    (None, 109, 109, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
__________

In [18]:
start_time = time.time()

model1.fit(X_train, Y_train, 
         batch_size=batch_size, epochs=epochs, 
         callbacks=[early_stopping],
         validation_data=(X_valid, Y_valid), shuffle=True)

end_time = time.time()
print ('The time for fitting training data into model 1 is {}'.format(end_time-start_time))

Train on 1184 samples, validate on 297 samples
Epoch 1/50
1184/1184 [==============================] - 376s 318ms/step - loss: 1.0104 - acc: 0.5312 - val_loss: 1.0173 - val_acc: 0.5118
Epoch 2/50
1184/1184 [==============================] - 357s 302ms/step - loss: 1.0059 - acc: 0.5270 - val_loss: 1.0164 - val_acc: 0.5118
Epoch 3/50
1184/1184 [==============================] - 325s 274ms/step - loss: 0.9942 - acc: 0.5312 - val_loss: 1.0145 - val_acc: 0.5118
Epoch 4/50
1184/1184 [==============================] - 299s 252ms/step - loss: 0.9832 - acc: 0.5321 - val_loss: 1.0087 - val_acc: 0.5118
Epoch 5/50
1184/1184 [==============================] - 282s 238ms/step - loss: 0.9607 - acc: 0.5312 - val_loss: 0.9629 - val_acc: 0.5118
The time for fitting training data into model 1 is 1639.1563205718994


### Very Deep Convolutional NetWorks for Large-Scale Image Recognition (VGG16 Model)

In [23]:
from keras.applications.vgg16 import VGG16
from keras.models import Model 

vgg16_model = VGG16(weights = 'imagenet', include_top=False, 
                   input_shape=(img_w, img_h, 3))
top_model = Sequential()
top_model.add(Flatten(input_shape = vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(3,activation='softmax'))

model2 = Model(input=vgg16_model.input, output=top_model(vgg16_model.output))
model2.compile(loss='categorical_crossentropy', 
             optimizer=sgd,
             metrics = ['accuracy'])

/home/ives/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  if sys.path[0] == '':


In [24]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [25]:
start_time = time.time()
model2.fit(X_train, Y_train, 
         batch_size=batch_size, epochs=epochs, 
         callbacks=[early_stopping], 
         validation_data=(X_valid, Y_valid), shuffle=True)

end_time = time.time()
print ('The time for fitting training data into model 2 is {}'.format(end_time-start_time))
model2.save_weights('model2.h5')

Train on 1184 samples, validate on 297 samples
Epoch 1/50
1184/1184 [==============================] - 4532s 4s/step - loss: 12.7385 - acc: 0.1934 - val_loss: 13.1875 - val_acc: 0.1818
Epoch 2/50
1184/1184 [==============================] - 4646s 4s/step - loss: 13.3410 - acc: 0.1723 - val_loss: 13.1875 - val_acc: 0.1818
Epoch 3/50
1184/1184 [==============================] - 4649s 4s/step - loss: 13.4499 - acc: 0.1655 - val_loss: 13.1875 - val_acc: 0.1818
Epoch 4/50
1184/1184 [==============================] - 5276s 4s/step - loss: 13.4635 - acc: 0.1647 - val_loss: 13.1875 - val_acc: 0.1818
Epoch 5/50
1184/1184 [==============================] - 5414s 5s/step - loss: 13.3818 - acc: 0.1698 - val_loss: 13.1875 - val_acc: 0.1818
The time for fitting training data into model 2 is 24517.816619873047


### ResNet Model

In [26]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model 

resnet50_model = ResNet50(weights = 'imagenet', include_top=False, 
                   input_shape=(img_w, img_h, 3))
add_model = Sequential()
add_model.add(Flatten(input_shape =resnet50_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(3,activation='softmax'))

model3 = Model(input=resnet50_model.input, output=add_model(resnet50_model.output))
model3.compile(loss='categorical_crossentropy', 
             optimizer=sgd,
             metrics = ['accuracy'])

/home/ives/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/ives/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  if sys.path[0] == '':


In [27]:
start_time = time.time()
model3.fit(X_train, Y_train, 
         batch_size=batch_size, epochs=epochs, 
         callbacks=[early_stopping], 
         validation_data=(X_valid, Y_valid), shuffle=True)

end_time = time.time()
print ('The time for fitting training data into model 3 is {}'.format(end_time-start_time))

model3.save_weights('model3.h5')

Train on 1184 samples, validate on 297 samples
Epoch 1/50
1184/1184 [==============================] - 3016s 3s/step - loss: 7.5120 - acc: 0.5270 - val_loss: 7.8691 - val_acc: 0.5118
Epoch 2/50
1184/1184 [==============================] - 2855s 2s/step - loss: 7.5554 - acc: 0.5312 - val_loss: 7.8691 - val_acc: 0.5118
Epoch 3/50
1184/1184 [==============================] - 2537s 2s/step - loss: 7.5554 - acc: 0.5312 - val_loss: 7.8691 - val_acc: 0.5118
Epoch 4/50
1184/1184 [==============================] - 2509s 2s/step - loss: 7.5554 - acc: 0.5312 - val_loss: 7.8691 - val_acc: 0.5118
Epoch 5/50
1184/1184 [==============================] - 2553s 2s/step - loss: 7.5554 - acc: 0.5312 - val_loss: 7.8691 - val_acc: 0.5118
The time for fitting training data into model 3 is 13474.341937065125


### TUNING ON MODEL 1 USING DIFFERENT COMBINATION OF LOSS FUNCTION AND OPTIMIZER

In [28]:
train_paths_selected = np.random.choice(train_paths, size=400)
test_paths_selected = np.random.choice(test_paths, size=100)

In [29]:
train_data_selected, train_target_selected, train_id_selected = load_train(train_paths_selected)
test_data_selected, test_id_selected = load_test(test_paths_selected)

Training data load time: 79.5964286327362
Testing data load time: 19.098011016845703


In [30]:
# Convert train_target to one-hot-encoding before fitting into the model
train_label_selected = [int(x.split('_')[1]) for x in train_target_selected]
train_label_selected = np_utils.to_categorical(train_label_selected)
train_label_selected = np.transpose(train_label_selected)
train_label_selected = np.transpose(train_label_selected[~(train_label_selected==0).all(axis=1)])

In [31]:
train_data_selected = normalize_data(train_data_selected)
print ('Shape of the training data is ', train_data_selected.shape)

Shape of the training data is  (400, 224, 224, 3)


In [32]:
test_data_selected = normalize_data(test_data_selected)
print ('Shape of the training data is ', test_data_selected.shape)

Shape of the training data is  (100, 224, 224, 3)


In [33]:
# Create validation split 
X_train_selected, X_valid_selected, Y_train_selected, Y_valid_selected = train_test_split(train_data_selected, 
                                                                                          train_label_selected, 
                                                                                          test_size=0.2, 
                                                                                          random_state=22)

In [34]:
def build_model(loss_function, optimizer):
    model = Sequential()
    kernel_size=(3,3)
    data_size=(224,224,3)
    for i in range(5, 8):
        model.add(Conv2D(2^i, kernel_size, input_shape=data_size, data_format='channels_last'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss=loss_function, optimizer = optimizer, 
                 metrics=['accuracy'])
    return model

In [35]:
from keras import optimizers
loss_functions = ['mean_squared_error', 
                 'categorical_crossentropy', 
                 'binary_crossentropy']

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adadelta = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

optimizers = [sgd, adadelta, adam]

In [36]:
combinations = [loss_functions, optimizers]
import itertools
combinations = list(itertools.product(*combinations))
print ('The number of combinations is ', len(combinations))

The number of combinations is  9


In [37]:
#Mew Early stopping callback
patience = 10
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0.01, 
                              patience = patience, verbose=0, mode='auto')

In [ ]:
for i in range(len(combinations)):
    loss_function = combinations[i][0]
    optimizer = combinations[i][1]
    print ('Loss function: ', loss_function)
    print ('Optimizer: ', optimizer)
    start_time = time.time()
    model = build_model(loss_function, optimizer)
    modelcheckpoint = ModelCheckpoint('Model1-{}.h5'.format(str(i)), 
                                            monitor='val_acc', 
                                            verbose=1, save_best_only=True,  
                                            mode='auto', period=1)
    model.fit(X_train, Y_train, 
              batch_size=batch_size, 
             epochs=epochs, 
             callbacks=[modelcheckpoint, early_stopping], 
             validation_data=[X_valid, Y_valid])
    print ('Training time for this model is {}'.format(time.time()-start_time))
    print ('')
    print ('')

Loss function:  mean_squared_error
Optimizer:  <keras.optimizers.SGD object at 0x2b4edb69c278>
Train on 1184 samples, validate on 297 samples
Epoch 1/50
1184/1184 [==============================] - 34s 29ms/step - loss: 0.2042 - acc: 0.5152 - val_loss: 0.2118 - val_acc: 0.5118

Epoch 00001: val_acc improved from -inf to 0.51178, saving model to Model1-0.h5
Epoch 2/50
1184/1184 [==============================] - 34s 29ms/step - loss: 0.2010 - acc: 0.5304 - val_loss: 0.2088 - val_acc: 0.5118

Epoch 00002: val_acc did not improve from 0.51178
Epoch 3/50
1184/1184 [==============================] - 37s 31ms/step - loss: 0.1992 - acc: 0.5321 - val_loss: 0.2068 - val_acc: 0.5118

Epoch 00003: val_acc did not improve from 0.51178
Epoch 4/50
1184/1184 [==============================] - 42s 35ms/step - loss: 0.1968 - acc: 0.5312 - val_loss: 0.2040 - val_acc: 0.5118

Epoch 00004: val_acc did not improve from 0.51178
Epoch 5/50
1184/1184 [==============================] - 49s 41ms/step - loss: 0.

1184/1184 [==============================] - 39s 33ms/step - loss: 0.1623 - acc: 0.6267 - val_loss: 0.2017 - val_acc: 0.5320

Epoch 00017: val_acc did not improve from 0.54209
Epoch 18/50
1184/1184 [==============================] - 39s 33ms/step - loss: 0.1542 - acc: 0.6529 - val_loss: 0.2054 - val_acc: 0.5286

Epoch 00018: val_acc did not improve from 0.54209
Epoch 19/50
1184/1184 [==============================] - 40s 34ms/step - loss: 0.1494 - acc: 0.6639 - val_loss: 0.2054 - val_acc: 0.5084

Epoch 00019: val_acc did not improve from 0.54209
Epoch 20/50
1184/1184 [==============================] - 40s 34ms/step - loss: 0.1449 - acc: 0.6900 - val_loss: 0.2092 - val_acc: 0.5152

Epoch 00020: val_acc did not improve from 0.54209
Epoch 21/50
1184/1184 [==============================] - 37s 31ms/step - loss: 0.1424 - acc: 0.6765 - val_loss: 0.2065 - val_acc: 0.5152

Epoch 00021: val_acc did not improve from 0.54209
Epoch 22/50
1184/1184 [==============================] - 38s 32ms/step -

In [56]:
from keras.weights import load_weights
model = load_weights('best_model1.hdf5')

ModuleNotFoundError: No module named 'keras.weights'